In [1]:
import sys
import os
import importlib
from collections import Counter
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# --- 1. Path Setup ---
# Absolute path to repo root (adjust if necessary)
repo_root = "/files/pixlball"
if repo_root not in sys.path:
    sys.path.insert(0, repo_root) 

# --- 2. Project Module Imports ---
# Import all project modules using clean names
import src.config as config
import src.dataset as dataset
import src.train as train
import src.evaluate as evaluate
import src.data as data
import src.losses as losses
import src.model as model
import src.utils as utils

# --- 3. Module Reloading (CRITICAL for Notebook Development) ---
# Reload dependencies in order: Config/Utils -> Data/Losses/Model -> Train/Dataset/Evaluate
importlib.reload(config)
importlib.reload(utils)
importlib.reload(data)
importlib.reload(model)
importlib.reload(losses) 
importlib.reload(dataset)
importlib.reload(train)
importlib.reload(evaluate)

# --- 4. Direct Imports (For clean code in subsequent cells) ---
# Import essential classes and functions needed for the pipeline steps

# Configuration
from src.config import DEVICE 

# Data/Dataset Classes
from src.dataset import PitchDatasetMultiTask, TemporalPitchDataset, ContextPitchDatasetMultiTask, FusionPitchDataset

# Training Functions
from src.train import train_model_base, train_model_lstm, train_model_context, train_model_lstm_fused

# Evaluation/Helpers
from src.evaluate import evaluate_model_base, evaluate_model_lstm, evaluate_model_context, evaluate_model_lstm_fused
from src.losses import get_model_criteria
from src.model import TinyCNN_MultiTask, HybridCNN_LSTM, TinyCNN_LSTM_Fused
from src.utils import get_sequence_lengths

# --- Final Check ---
print(f"Using device: {DEVICE}")

Using device: cpu


In [2]:
data_events = pd.read_parquet(os.path.join(repo_root, "data", "events_data.parquet"), engine="fastparquet")
data_360 = pd.read_parquet(os.path.join(repo_root, "data", "sb360_data.parquet"), engine="fastparquet")

In [3]:
admin_events = [
        'Starting XI', 'Half Start', 'Half End', 'Player On', 'Player Off',
        'Substitution', 'Tactical Shift', 'Referee Ball-Drop', 'Injury Stoppage',
        'Bad Behaviour', 'Shield'
    ]

cleaned_df = data.drop_events(data_events, rows_to_drop=admin_events)

1278 events.


In [4]:
# -----------------------------
# Example usage
# -----------------------------
columns_to_drop = ['clearance_body_part',
                   'clearance_head',
                   'clearance_left_foot',
                   'clearance_other',
                   'clearance_right_foot',
                   'shot_technique',
                   'substitution_replacement_id',
                   'substitution_replacement',
                   'substitution_outcome',
                   'shot_saved_off_target',
                   'pass_miscommunication',
                   'goalkeeper_shot_saved_off_target',
                   'goalkeeper_punched_out',
                   'shot_first_time',
                   'shot_first_time',
                   'shot_body_part',
                   'related_events',
                   'pass_shot_assist', 
                   'pass_straight', 
                   'pass_switch', 
                   'pass_technique', 
                   'pass_through_ball',
                   'goalkeeper_body_part',
                   'goalkeeper_end_location', 
                   'goalkeeper_outcome', 
                   'goalkeeper_position', 
                   'goalkeeper_technique', 
                   'goalkeeper_type', 
                   'goalkeeper_penalty_saved_to_post', 
                   'goalkeeper_shot_saved_to_post', 
                   'goalkeeper_lost_out', 
                   'goalkeeper_Clear', 
                   'goalkeeper_In Play Safe',
                   'shot_key_pass_id',
                   'shot_one_on_one',
                   'shot_end_location',
                   'shot_type',
                   'pass_angle',
                   'pass_body_part',
                   'pass_type',
                   'pass_length',
                   'pass_outswinging',
                   'pass_inswinging',
                   'pass_cross', 
                   'pass_cut_back', 
                   'pass_deflected', 
                   'pass_goal_assist', 
                   'pass_recipient', 
                   'pass_recipient_id', 
                   'pass_assisted_shot_id', 
                   'pass_no_touch', 
                   'pass_end_location', 
                   'pass_aerial_won',
                   'pass_height',
                   'substitution_outcome_id',
                   'tactics',
                   'block_deflection',
                   'dribble_no_touch',
                   'shot_open_goal', 
                   'shot_saved_to_post',
                   'shot_redirect', 
                   'shot_follows_dribble',
                   'period',
                   'injury_stoppage_in_chanin',
                   'block_save_block',
                   'ball recovery_offensive',


                   ]
cleaned_df = data.drop_columns(cleaned_df, columns_to_drop)

# add lookahead outcome
df_with_targets = data.assign_lookahead_outcomes(cleaned_df, lookahead=6)


counts of each outcome nn_target
Keep Possession    71251
Lose Possession    28252
Shot                4830
Name: count, dtype: int64


# Prepare 360 Data

In [5]:
df_360 = data.assign_grid_cells(data_360)
nn_final = data.aggregate_nn_layers_vectorized(df_360)

# Finalize NN Df

In [6]:
nn_dataset = data.prepare_nn_dataset(df_with_targets, nn_final, target_cols=['nn_target', 'goal_flag'], context_cols = True, keep_context_ids = True ) # adjust cols depending on model

# Neural Network final Data Prep

In [7]:
context_cols = [
    'under_pressure', 
    'counterpress', 
    'dribble_nutmeg'
]

# Impute NaN values with 0.0 (float)
# This assumes NaN means the event was NOT under pressure, NOT a counterpress, etc.
nn_dataset[context_cols] = nn_dataset[context_cols].fillna(0.0)


target_map = {"Keep Possession": 0, "Lose Possession": 1, "Shot": 2}

# Apply mapping
nn_dataset['nn_target_int'] = nn_dataset['nn_target'].map(target_map)

# Check
print(nn_dataset[['nn_target', 'nn_target_int']].head())

         nn_target  nn_target_int
0  Keep Possession              0
1  Keep Possession              0
2  Keep Possession              0
3  Keep Possession              0
4  Keep Possession              0


# The Goal Multi Task CNN

In [8]:
# ------------------------------------
# 1. Define input columns & targets
# ------------------------------------
# This assumes nn_dataset is already loaded and processed in previous cells.
layer_columns = ["ball_layer", "teammates_layer", "opponents_layer"]

# Ensure labels are in the correct format
event_targets = nn_dataset['nn_target_int'].values   # 0=keep, 1=lose, 2=shot (int)
# CRITICAL: Goal flags must be float for BCEWithLogitsLoss
goal_flags = nn_dataset['goal_flag'].values.astype(np.float32) 

# ------------------------------------
# 3. Compute class weights and positive weight
# ------------------------------------

# A. Event Weights (Multi-Class) - For CrossEntropyLoss
event_counts = Counter(event_targets)
total_events = len(event_targets)

# Using inverse frequency: total / count
class_weights_event = torch.tensor(
    [total_events / event_counts.get(c, 1) for c in range(len(event_counts))],
    dtype=torch.float32
).to(DEVICE)

# B. Goal Positive Weight (Binary) - For BCEWithLogitsLoss
goal_counts = Counter(goal_flags)
# CRITICAL: pos_weight = (Number of Negative Samples) / (Number of Positive Samples)
# Here: pos_weight = (Number of No Goals) / (Number of Goals)
goal_pos_weight = torch.tensor(
    goal_counts.get(0.0, 1) / goal_counts.get(1.0, 1),
    dtype=torch.float32
).to(DEVICE)

print(f"Goal Positive Weight (0/1 ratio): {goal_pos_weight.item():.2f}")


Goal Positive Weight (0/1 ratio): 168.51


# Preparing the Context CNN

In [9]:
# --- 1. Define Context Features ---
# Based on your previous steps, you used 3 binary features.
NUM_CONTEXT_FEATURES = 3 

FINAL_CONTEXTUAL_FEATURES = [
    'under_pressure', 
    'counterpress', 
    'dribble_nutmeg'
]

nn_dataset = data.prepare_nn_dataset(df_with_targets, nn_final, target_cols=['nn_target', 'goal_flag'], context_cols = True, keep_context_ids = True )

target_map = {"Keep Possession": 0, "Lose Possession": 1, "Shot": 2}

# Apply mapping
nn_dataset['nn_target_int'] = nn_dataset['nn_target'].map(target_map)

# Ensure the context DataFrame is prepared (imputed NaN with 0.0)
context_df = nn_dataset[FINAL_CONTEXTUAL_FEATURES].copy().fillna(0.0) 

# Re-check the number of features just to be safe
print(f"Number of Contextual Features being used: {context_df.shape[1]}") 
print(f"Contextual Features Head:\n{context_df.head()}")

# --- 2. Define Inputs and Targets (Same as before) ---
#layer_columns = ["ball_layer", "teammates_layer", "opponents_layer"]
#event_targets = nn_dataset['nn_target_int'].values
#goal_flags = nn_dataset['goal_flag'].values.astype(np.float32) 

# --- 3. Create Contextual Dataset Instance ---
# CRITICAL: PitchDatasetMultiTask must be updated to accept the 4th input
train_dataset_context = ContextPitchDatasetMultiTask(
    nn_layers_df=nn_dataset[layer_columns], 
    event_targets=event_targets, 
    goal_flags=goal_flags,
    contextual_features_df=context_df # NEW ARGUMENT
)

print(f"Contextual Dataset Size: {len(train_dataset_context)}")
print("Contextual Dataset ready.")

Number of Contextual Features being used: 3
Contextual Features Head:
   under_pressure  counterpress  dribble_nutmeg
0             0.0           0.0             0.0
1             0.0           0.0             0.0
2             1.0           0.0             0.0
3             1.0           0.0             0.0
4             0.0           0.0             0.0
Contextual Dataset Size: 90690
Contextual Dataset ready.


In [10]:
# Assuming event_class_weights and goal_pos_weight are defined from previous cells

print("Starting training for Contextual CNN Baseline...")

context_baseline_model = train_model_context(
    dataset=train_dataset_context, 
    event_class_weights=class_weights_event, # Use your calculated weights
    goal_pos_weight=goal_pos_weight,         # Use your calculated pos_weight
    num_context_features=NUM_CONTEXT_FEATURES
)

print("\nContextual CNN Training complete.")

Context CNN Epoch 1:   0%|          | 25/9069 [00:00<00:37, 243.40it/s, event_loss=1.07, loss=1.07, shot_loss=0]    

Starting training for Contextual CNN Baseline...


Context CNN Epoch 2: 100%|██████████| 9069/9069 [00:36<00:00, 250.01it/s, event_loss=0.828, loss=0.828, shot_loss=0]   


Contextual CNN Training complete.


In [11]:
# Assuming evaluate_model_context is imported and available

print("\nEvaluating Contextual CNN Model...")

metrics = evaluate_model_context(
    model=context_baseline_model, 
    dataset=train_dataset_context # Evaluate on the contextual dataset
)

# You can now compare 'context_metrics' with your 'baseline_metrics'
# print(context_metrics)


Evaluating Contextual CNN Model...


ValueError: zero-dimensional arrays cannot be concatenated